In [6]:
import pandas as pd
import requests

In [7]:
df=pd.read_csv("CountyRedfinDataRaw.gz", sep='\t')

In [8]:
df.columns


Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [9]:
df=df[df['property_type']=='All Residential']
#df[df['region']=='Orange County, CA']
attributes=['period_end','region','state','state_code','median_ppsf']
df=df[attributes]
df['region'] = df['region'].str.split(',').str[0]
df.rename(columns={'region':'county'}, inplace=True)
df

,period_end,county,state,state_code,median_ppsf
0,2017-04-30,Morgan County,Alabama,AL,75.845434
2,2018-02-28,Tulsa County,Oklahoma,OK,89.765580
11,2021-01-31,Hood River County,Oregon,OR,300.167926
13,2015-08-31,Lamar County,Mississippi,MS,93.454628
17,2018-11-30,Pasco County,Florida,FL,115.155545
...,...,...,...,...,...
563114,2016-06-30,Ferry County,Washington,WA,43.217599
563116,2014-07-31,Newaygo County,Michigan,MI,75.000000
563117,2013-03-31,Gonzales County,Texas,TX,83.404488
563119,2016-06-30,Livingston Parish,Louisiana,LA,108.063175


COVID CASES

In [74]:
from io import StringIO
API_KEY="2c581a67ac0045ff88f776e32dff0564"
response = requests.get(f"https://api.covidactnow.org/v2/counties.csv?apiKey={API_KEY}")
covid_data = StringIO(response.text) 
coviddf=pd.read_csv(covid_data)

In [75]:
coviddf.columns

Index(['fips', 'country', 'state', 'county', 'level', 'lat', 'locationId',
       'long', 'population', 'metrics.testPositivityRatio',
       'metrics.testPositivityRatioDetails.source', 'metrics.caseDensity',
       'metrics.contactTracerCapacityRatio', 'metrics.infectionRate',
       'metrics.infectionRateCI90', 'unused1', 'unused2',
       'metrics.icuCapacityRatio', 'riskLevels.overall',
       'riskLevels.testPositivityRatio', 'riskLevels.caseDensity',
       'riskLevels.contactTracerCapacityRatio', 'riskLevels.infectionRate',
       'unused3', 'riskLevels.icuCapacityRatio', 'actuals.cases',
       'actuals.deaths', 'actuals.positiveTests', 'actuals.negativeTests',
       'actuals.contactTracers', 'actuals.hospitalBeds.capacity',
       'actuals.hospitalBeds.currentUsageTotal',
       'actuals.hospitalBeds.currentUsageCovid', 'unused4',
       'actuals.icuBeds.capacity', 'actuals.icuBeds.currentUsageTotal',
       'actuals.icuBeds.currentUsageCovid', 'unused5', 'actuals.newCases',

In [76]:
coviddf=coviddf[["state","county","actuals.cases","metrics.vaccinationsCompletedRatio","riskLevels.overall"]]

In [77]:
riskLevelMapper={0:"Low",1:"Medium",2:"High",3:"Critical",4:"Unknown",5:"Extreme"}
coviddf['Risk']=coviddf['riskLevels.overall'].apply(lambda value: riskLevelMapper[value])
coviddf.rename(columns={'state':'state_code','actuals.cases':'Daily Cases','metrics.vaccinationsCompletedRatio':' Percentage Fully Vaccinated','riskLevels.overall':'Covid Risk Index'}, inplace=True)


In [79]:
coviddf[coviddf['state_code']=='CA']

,state_code,county,Daily Cases,Percentage Fully Vaccinated,Risk Index,Risk
186,CA,Alameda County,259290,0.804,3,Critical
187,CA,Alpine County,126,0.636,2,High
188,CA,Amador County,8840,0.519,3,Critical
189,CA,Butte County,38643,0.522,3,Critical
190,CA,Calaveras County,7495,0.526,3,Critical
191,CA,Colusa County,4444,0.590,3,Critical
192,CA,Contra Costa County,195310,0.811,3,Critical
193,CA,Del Norte County,5820,0.466,5,Extreme
194,CA,El Dorado County,29173,0.602,3,Critical
195,CA,Fresno County,244801,0.597,3,Critical


In [82]:
dfinal = df.merge(coviddf, on=["state_code","county"], how = 'left')


In [83]:
dfinal

,period_end,county,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_ppsf,median_list_price,homes_sold,homes_sold_mom,homes_sold_yoy,median_dom,Daily Cases,Percentage Fully Vaccinated,Risk Index,Risk
0,2017-04-30,Morgan County,Alabama,AL,All Residential,-1,120500.0,-0.076628,75.845434,147900.0,106.0,-0.109244,-0.131148,101.0,35533.0,0.430,3.0,Critical
1,2014-12-31,Hennepin County,Minnesota,MN,Multi-Family (2-4 Unit),4,203000.0,-0.171429,86.633333,210000.0,57.0,0.628571,-0.109375,76.0,291774.0,0.729,3.0,Critical
2,2018-02-28,Tulsa County,Oklahoma,OK,All Residential,-1,167500.0,0.021341,89.765580,179700.0,665.0,0.136752,0.084829,69.0,167129.0,0.605,3.0,Critical
3,2017-08-31,Crow Wing County,Minnesota,MN,Condo/Co-op,3,125000.0,-0.137931,147.076613,209900.0,3.0,2.000000,NaN,107.0,15620.0,0.558,3.0,Critical
4,2019-05-31,Tate County,Mississippi,MS,Single Family Residential,6,165000.0,-0.144635,85.000000,181444.0,19.0,0.000000,-0.050000,117.0,7222.0,0.442,3.0,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-31,Gonzales County,Texas,TX,All Residential,-1,291500.0,2.285210,83.404488,65000.0,2.0,0.000000,-0.333333,458.0,4417.0,0.486,1.0,Medium
563118,2013-09-30,Valley County,Idaho,ID,Townhouse,13,221500.0,1.605882,131.923764,NaN,1.0,0.000000,0.000000,170.0,2445.0,0.613,3.0,Critical
563119,2016-06-30,Livingston Parish,Louisiana,LA,All Residential,-1,173990.0,0.030136,108.063175,181295.0,197.0,0.165680,0.026042,50.0,36759.0,0.391,2.0,High
563120,2017-09-30,Gloucester County,Virginia,VA,Single Family Residential,6,170000.0,0.013050,113.502506,185000.0,9.0,0.285714,-0.181818,91.0,7166.0,0.659,3.0,Critical


In [85]:
dfinal.to_csv("HousingDatasetwithCovidData.csv",index=False)

In [98]:
taxesDf=pd.read_csv("TaxBurdenDataset.txt")
taxesDf.rename(columns={'State':'state'}, inplace=True)


In [94]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [99]:
#taxesDf['state_code']=taxesDf['state'].apply(lambda value: us_state_to_abbrev[value])
taxesDf

,Cost of Living Rank,state,Total Tax Burden
0,1,New York,12.79%
1,2,Hawaii,12.19%
2,3,Vermont,10.75%
3,4,Maine,10.50%
4,5,Connecticut,10.44%
5,6,Minnesota,9.99%
6,7,New Jersey,9.98%
7,8,Rhode Island,9.69%
8,9,Illinois,9.52%
9,10,California,9.48%


In [100]:
dfinal1 = dfinal.merge(taxesDf, on=["state"], how = 'left')
dfinal1

,period_end,county,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_ppsf,median_list_price,homes_sold,homes_sold_mom,homes_sold_yoy,median_dom,Daily Cases,Percentage Fully Vaccinated,Risk Index,Risk,Cost of Living Rank,Total Tax Burden
0,2017-04-30,Morgan County,Alabama,AL,All Residential,-1,120500.0,-0.076628,75.845434,147900.0,106.0,-0.109244,-0.131148,101.0,35533.0,0.430,3.0,Critical,40.0,7.45%
1,2014-12-31,Hennepin County,Minnesota,MN,Multi-Family (2-4 Unit),4,203000.0,-0.171429,86.633333,210000.0,57.0,0.628571,-0.109375,76.0,291774.0,0.729,3.0,Critical,6.0,9.99%
2,2018-02-28,Tulsa County,Oklahoma,OK,All Residential,-1,167500.0,0.021341,89.765580,179700.0,665.0,0.136752,0.084829,69.0,167129.0,0.605,3.0,Critical,44.0,7.13%
3,2017-08-31,Crow Wing County,Minnesota,MN,Condo/Co-op,3,125000.0,-0.137931,147.076613,209900.0,3.0,2.000000,NaN,107.0,15620.0,0.558,3.0,Critical,6.0,9.99%
4,2019-05-31,Tate County,Mississippi,MS,Single Family Residential,6,165000.0,-0.144635,85.000000,181444.0,19.0,0.000000,-0.050000,117.0,7222.0,0.442,3.0,Critical,15.0,8.99%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-31,Gonzales County,Texas,TX,All Residential,-1,291500.0,2.285210,83.404488,65000.0,2.0,0.000000,-0.333333,458.0,4417.0,0.486,1.0,Medium,30.0,8.19%
563118,2013-09-30,Valley County,Idaho,ID,Townhouse,13,221500.0,1.605882,131.923764,NaN,1.0,0.000000,0.000000,170.0,2445.0,0.613,3.0,Critical,36.0,7.88%
563119,2016-06-30,Livingston Parish,Louisiana,LA,All Residential,-1,173990.0,0.030136,108.063175,181295.0,197.0,0.165680,0.026042,50.0,36759.0,0.391,2.0,High,19.0,8.82%
563120,2017-09-30,Gloucester County,Virginia,VA,Single Family Residential,6,170000.0,0.013050,113.502506,185000.0,9.0,0.285714,-0.181818,91.0,7166.0,0.659,3.0,Critical,37.0,7.86%


In [101]:
dfinal1.to_csv("DataWCovidTaxes.csv",index=False)

In [104]:
ageDf=pd.read_csv("averageAgeRawDataset.csv")
ageDf.rename(columns={'State':'state'}, inplace=True)
ageDf

,state,MedianAge
0,Maine,45
1,West Virginia,43
2,Vermont,43
3,New Hampshire,43
4,Florida,42
5,Pennsylvania,41
6,Delaware,41
7,Connecticut,41
8,Wisconsin,40
9,Rhode Island,40


# dfinalAge = dfinal1.merge(ageDf, on=["state"], how = 'left')
dfinalAge

In [106]:
dfinalAge.to_csv("HousingRecommenderFinalDataset.csv",index=False)